In [1]:
from torch.utils.data import Dataset
import pickle
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, data_files, transform=None):
        """
        Args:
            data_files (list): List of file paths to the data batches.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data = []
        self.labels = []
        self.transform = transform

        # Load data from all files
        for file in data_files:
            with open(file, 'rb') as f:
                batch = pickle.load(f, encoding='bytes')
                self.data.extend(batch[b'data'])  # Assuming data is stored under the key b'data'
                self.labels.extend(batch[b'labels'])  # Assuming labels are stored under the key b'labels'

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]
        
        # 检查原始数据的形状
        # print(f"Original data shape: {image.shape}, {label}")

        image = image.reshape(32, 32, 3).astype('uint8')
        image = Image.fromarray(image)  # 转换为 PIL 图像

        # 检查转换后的形状
        # print(f"Transformed image shape: {image.shape}")

        if self.transform:
            image = self.transform(image)
        return image, label

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LeNetPlus(nn.Module):  
    def __init__(self):  
        super().__init__()  
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)  # 更宽的第一层  
        self.bn1 = nn.BatchNorm2d(32)  
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)  
        self.bn2 = nn.BatchNorm2d(64)  
        self.fc1 = nn.Linear(64 * 8 * 8, 256)        # 调整全连接层尺寸  
        self.fc2 = nn.Linear(256, 10)  
  
    def forward(self, x):  
        x = F.relu(self.bn1(self.conv1(x)))  
        x = F.max_pool2d(x, 2)  
        x = F.relu(self.bn2(self.conv2(x)))  
        x = F.max_pool2d(x, 2)  
        x = x.view(-1, 64 * 8 * 8)  
        x = F.relu(self.fc1(x))  
        x = self.fc2(x)  
        return x

class LeNet(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5, stride=1, padding=2)  # 输入通道为3（RGB），输出通道为6
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # 池化层
        self.conv2 = nn.Conv2d(6, 16, kernel_size=3)  # 第二个卷积层
        self.fc1 = nn.Linear(16 * 7 * 7, 120)  # 全连接层1
        self.fc2 = nn.Linear(120, 84)  # 全连接层2
        self.fc3 = nn.Linear(84, num_classes)  # 输出层

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 卷积1 -> ReLU -> 池化
        x = self.pool(F.relu(self.conv2(x)))  # 卷积2 -> ReLU -> 池化
        x = x.view(-1, 16 * 7 * 7)  # 展平操作
        x = F.relu(self.fc1(x))  # 全连接层1 -> ReLU
        x = F.relu(self.fc2(x))  # 全连接层2 -> ReLU
        x = self.fc3(x)  # 输出层
        return x

def evaluate(model, test_loader, device):
    """评估模型在测试集上的准确率"""
    model.eval()  # 确保模型处于评估模式
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model.forward(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

def train_model(model, train_loader, criterion, optimizer, num_epochs, device, test_loader=None):
    for epoch in range(num_epochs):
        epoch_loss = 0 
        model.train()  # 确保模型处于训练模式
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # 将数据移动到设备
            optimizer.zero_grad()
            outputs = model.forward(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() 
        
        # print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss / len(train_loader):.4f}') 
        
        # 每 10 个 epoch 评估一次
        if (epoch + 1) % 10 == 0 and test_loader is not None:
            print(f"Evaluating at epoch {epoch+1}...")
            accuracy = evaluate(model, test_loader, device)
            print(f'Accuracy at epoch {epoch+1}: {accuracy:.2f}%')

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms

test_files = [
    '/Users/mini4chaos/Developer/cifar-10/data/test_batch'
]

data_files = [
    '/Users/mini4chaos/Developer/cifar-10/data/data_batch_1',
    '/Users/mini4chaos/Developer/cifar-10/data/data_batch_2',
    '/Users/mini4chaos/Developer/cifar-10/data/data_batch_3',
    '/Users/mini4chaos/Developer/cifar-10/data/data_batch_4',
    '/Users/mini4chaos/Developer/cifar-10/data/data_batch_5'
]

# transform = transforms.Compose([  
#     transforms.RandomHorizontalFlip(),  
#     transforms.ToTensor(),  
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),  
# ])

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    # transforms.RandomCrop(32, padding=4),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
# ])

batch_size = 32

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# 加载数据集
train_dataset = CustomDataset(data_files, transform=transform)
test_dataset = CustomDataset(test_files, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# for inputs, labels in train_loader:
#     print(f"Inputs shape: {inputs.shape}, Labels shape: {labels.shape}")
#     break

# 初始化模型、损失函数和优化器
model = LeNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 20
train_model(model, train_loader, criterion, optimizer, num_epochs, device, test_loader)

Using device: mps
Epoch [1/20], Loss: 1.8854
Epoch [2/20], Loss: 1.7090
Epoch [3/20], Loss: 1.6392
Epoch [4/20], Loss: 1.5806
Epoch [5/20], Loss: 1.5365
Epoch [6/20], Loss: 1.5082
Epoch [7/20], Loss: 1.4849
Epoch [8/20], Loss: 1.4593
Epoch [9/20], Loss: 1.4515
Epoch [10/20], Loss: 1.4330
Evaluating at epoch 10...
Accuracy at epoch 10: 47.49%
Epoch [11/20], Loss: 1.4249
Epoch [12/20], Loss: 1.4130
Epoch [13/20], Loss: 1.4027
Epoch [14/20], Loss: 1.4002
Epoch [15/20], Loss: 1.3831
Epoch [16/20], Loss: 1.3811
Epoch [17/20], Loss: 1.3755
Epoch [18/20], Loss: 1.3689
Epoch [19/20], Loss: 1.3703
Epoch [20/20], Loss: 1.3622
Evaluating at epoch 20...
Accuracy at epoch 20: 49.21%
